In [3]:
#imports
import pandas as pd
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.decomposition import PCA
from pickle import dump,load
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression

def cohen_kappa_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    return cohen_kappa_score(y, y_pred)

# Features

* Freezing (precipitation + temp < 32)

* Weekend (F-M)

* Peak times

* Tailnum -> seats

* Recent (previous) delays by carrier

* Recent delays by origin/dest airport

* Plane history

* 6-9 PM most delays https://www.rd.com/article/avoid-delays-best-time-day-to-fly/

* Daily flight count

# Process

* Categorize as delayed/not, aim to capture as many delays as possible (recall>accuracy), accuracy can be determined in next

* Categorize delay severity based on prior categorized delays (regression?)

* Find agreement between models





In [4]:
#convert numeric response to categorical
def categorize_delays(delays):
    result = np.where(delays < 30, 'ontime',
             np.where((delays >= 30) & (delays <= 120), 'minordelay',
             np.where(delays > 120, 'majordelay', delays)))
    return result


#match columns of original dataset
def match_cols(original, new):
    original_cols = original.columns

    for col in original_cols:
        if col not in new.columns:
            new[col] = 0

    new = new[original_cols]
    return new


#returns x,y
def get_data(path):
    flights = pd.read_csv(path)
    planes = pd.read_csv('planes.csv')
    weather = pd.read_csv('weather.csv')
    modeldata = pd.read_csv('flight_data_full.csv') #used for column matching

    #impute weather
    weather_orig = weather['origin']
    weather = weather.drop(['wind_gust','origin','time_hour','year'],axis=1)
    imputer = IterativeImputer(sample_posterior=True)
    weather = pd.DataFrame(imputer.fit_transform(weather), columns=weather.columns)
    weather.insert(0, 'origin', weather_orig)

    #impute airplanes
    planes = planes.drop('speed',axis=1)
    year_by_model = planes.groupby('model')['year'].first()
    planes['year'] = planes['year'].fillna(planes['model'].map(year_by_model)) #still some missing... use median
    planes['year'] = planes['year'].fillna(planes['year'].median())


    ##### New variables #####
    #delay severity
    flights['delay_severity'] = categorize_delays(flights['dep_delay'])

    #existance of a delay
    flights['is_delayed'] = np.where(flights['delay_severity'] == 'ontime', 0, 1)

    #snowing category
    weather['snowing'] = (weather['precip'] > 0) & (weather['temp'] <= 32).astype(int)

    #day of week + weekend category (F-M)
    flights['date'] = pd.to_datetime(flights[['year', 'month', 'day']])
    flights['day_of_week'] = flights['date'].dt.day_name()
    flights['is_weekend'] = flights['day_of_week'].isin(['Friday', 'Saturday', 'Sunday', 'Monday']).astype(int)

    #peak dates (Thanksgiving (11/28), Christmas, Memorial Day (5/27), July Fourth, and Labor Day(9/2)) pm 5 days
    peak_dates = pd.to_datetime(['2013-11-28', '2013-12-25', '2013-07-04', '2013-05-27', '2013-09-02'])

    peak_weeks = pd.DataFrame() #get 5 days before/after
    for date in peak_dates:
        date_range = pd.date_range(start=date - pd.Timedelta(days=5), 
                                end=date + pd.Timedelta(days=5))
        peak_weeks = pd.concat([peak_weeks, pd.DataFrame({'date': date_range})], ignore_index=True)
        
    flights['peak_week'] = flights['date'].isin(peak_weeks['date']).astype(int)

    #peak times (6PM-9PM)
    flights['peak_time'] = flights['hour'].between(18, 21)
    flights['peak_time'] = flights['peak_time'].astype(int)

    #prior airline, origin, and destination delays (takes 2 min to run)
    print('Getting new variables (1/3)')
    flights['date'] = pd.to_datetime(flights[['year', 'month', 'day', 'hour', 'minute']])

    flights['carrier_delay'] = flights.apply(
        lambda row: flights[(flights['carrier'] == row['carrier']) & 
                            (flights['date'] <= row['date']) & 
                            (flights['date'] > row['date'] - pd.Timedelta(hours=48))]['dep_delay'].mean(), axis=1)

    print('Getting new variables (2/3)')
    flights['origin_delay'] = flights.apply(
        lambda row: flights[(flights['origin'] == row['origin']) & 
                            (flights['date'] <= row['date']) & 
                            (flights['date'] > row['date'] - pd.Timedelta(hours=48))]['dep_delay'].mean(), axis=1)

    print('Getting new variables (3/3)')
    flights['dest_delay'] = flights.apply(
        lambda row: flights[(flights['dest'] == row['dest']) & 
                            (flights['date'] <= row['date']) & 
                            (flights['date'] > row['date'] - pd.Timedelta(hours=48))]['dep_delay'].mean(), axis=1)

    flights['carrier_delay'] = categorize_delays(flights['carrier_delay'])
    flights['carrier_delay'] = np.where(flights['carrier_delay'] == 'ontime', 0, 1)

    flights['origin_delay'] = categorize_delays(flights['origin_delay'])
    flights['origin_delay'] = np.where(flights['origin_delay'] == 'ontime', 0, 1)

    flights['dest_delay'] = categorize_delays(flights['dest_delay'])
    flights['dest_delay'] = np.where(flights['dest_delay'] == 'ontime', 0, 1)

    #number of flights leaving airport same day
    flights['flight_volume'] = flights.apply(
        lambda row: len(flights[(flights['origin'] == row['origin']) & 
                            (flights['year'] == row['year']) & 
                            (flights['month'] == row['month']) & 
                            (flights['day'] == row['day'])]),axis=1)

    #create final dataset
    flights = pd.merge(flights, weather, on=['month', 'day', 'hour', 'origin'])

    planes['year_manufactured'] = planes['year']
    planes = planes.drop('year',axis=1)
    flights = pd.merge(flights, planes, on='tailnum')

    #responses
    ys = flights[['dep_delay', 'delay_severity', 'is_delayed']]

    flights = flights.drop(['arr_time', 'arr_delay', 'flight','date','tailnum','air_time',
                            'year', 'month', 'day', 'dest', 'dep_time', 'dep_delay', 'delay_severity', 'is_delayed'],axis=1)
    
    modeldata = modeldata.drop(['Unnamed: 0','air_time','year', 'month', 'day', 'dest', 'dep_time', 'dep_delay', 'delay_severity', 'is_delayed'],axis=1)

    #predictors
    x = pd.get_dummies(flights,dtype=int)
    modeldata = pd.get_dummies(modeldata,dtype=int)

    #match columns to original data
    x = match_cols(modeldata, x)
    
    return x,ys

# Models

In [ ]:
##### DON'T RUN THIS #####
x,ys = get_data('flights_set0.csv')
y = ys['is_delayed']

Getting new variables (1/3)
Getting new variables (2/3)
Getting new variables (3/3)


In [ ]:
##### Data #####
data = pd.read_csv('flight_data_full.csv')
data = data.drop(['Unnamed: 0','air_time','year', 'month', 'day', 'dest', 'dep_time'],axis=1)

x = data.drop(['dep_delay', 'delay_severity', 'is_delayed'],axis=1)
x = pd.get_dummies(x,dtype=int)
y = data['is_delayed']

#split data
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=.7,random_state=764)

## Random Forest

* Kappa .457

* .79 recall on both

In [11]:
dc = DecisionTreeClassifier(class_weight='balanced')

param_grid = {
    'max_depth': [None, 2, 5, 7, 10],
    'min_samples_split': [2, 3, 5, 10, 20],
    'min_impurity_decrease': [0.0, 0.01, 0.1],
    'ccp_alpha': [0.0, 0.1, 0.2, 0.5, 0.7, 1.0] 
}

grid_search = GridSearchCV(estimator=dc, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0, scoring=cohen_kappa_scorer)

grid_search.fit(x_train, y_train)
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(x_test)
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

Best parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'min_impurity_decrease': 0.1, 'min_samples_split': 2}


In [ ]:
#Best parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'min_impurity_decrease': 0.1, 'min_samples_split': 2}

rf = RandomForestClassifier(class_weight='balanced',ccp_alpha=0,max_depth=None,min_impurity_decrease=0.1,min_samples_split=2)

param_grid = {
    'n_estimators': [50, 70, 90, 100, 150, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=0, scoring=cohen_kappa_scorer)

grid_search.fit(x_train, y_train)
best_tree = grid_search.best_estimator_
print(grid_search.best_score_)
y_pred = best_tree.predict(x_test)
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


In [12]:
#Best parameters: {'max_features': 'auto', 'n_estimators': 50}, kappa = .457

rf = RandomForestClassifier(class_weight='balanced',ccp_alpha=0,max_depth=None,min_samples_split=2,
                            max_features='sqrt',n_estimators=200)

rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print(classification_report(y_test, y_pred))

y_pred_prob = rf.predict_proba(x_test)[:, 1]
threshold = 0.15 #.79 both
y_pred_adjusted = (y_pred_prob >= threshold).astype(int)

print("Adjusted Threshold Classification Report:\n", classification_report(y_test, y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.89      0.99      0.93      5595
           1       0.82      0.34      0.49      1083

    accuracy                           0.88      6678
   macro avg       0.85      0.67      0.71      6678
weighted avg       0.88      0.88      0.86      6678

Adjusted Threshold Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.78      0.86      5595
           1       0.41      0.79      0.54      1083

    accuracy                           0.78      6678
   macro avg       0.68      0.78      0.70      6678
weighted avg       0.86      0.78      0.80      6678



## Testing data function on new data

In [78]:
x2,y2s = get_data('flights_set1.csv')

Getting new variables (1/3)
Getting new variables (2/3)
Getting new variables (3/3)


In [ ]:
#test new data
y2 = y2s['is_delayed']

y_pred = rf.predict(x2)
y_pred_prob = rf.predict_proba(x2)[:, 1]
threshold = 0.15 
y_pred_adjusted = (y_pred_prob >= threshold).astype(int)

print("Adjusted Threshold Classification Report:\n", classification_report(y2, y_pred_adjusted))

Adjusted Threshold Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.94     11778
           1       0.60      0.95      0.74      2125

    accuracy                           0.90     13903
   macro avg       0.80      0.92      0.84     13903
weighted avg       0.93      0.90      0.91     13903

